## Importação das bibliotecas

In [2]:
import basedosdados as bd
import pandas as pd

## Funções auxiliares

In [3]:
def generate_df(query):
    return bd.read_sql(query, billing_project_id="datario-eduardo-amorim")

## Carregando dataframes

In [7]:
#Carrega a tabela de Chamados abertos na data 01/04/2023 em um dataframe
query_chamados = """
    SELECT * FROM datario.administracao_servicos_publicos.chamado_1746
    WHERE DATE(data_inicio) = '2023-04-01';
"""

df_chamados = generate_df(query_chamados)

Downloading: 100%|██████████| 73/73 [00:00<00:00, 198.46rows/s]


In [8]:
#Carrega a tabela de Bairros em um dataframe 
query_bairros = """
    SELECT * FROM datario.dados_mestres.bairro;
"""

df_bairros = generate_df(query_bairros)

Downloading: 100%|██████████| 164/164 [00:11<00:00, 14.63rows/s]


## Questões referentes aos chamados nos bairros do RJ

Como o dataframe "df_chamados" já é fruto de uma consulta da tabela de chamados filtrada pela data desejada em todas as questões (01/04/2023), não será necessário aplicar uma filtragem por esta data em cada etapa.

### 1. Quantos chamados foram abertos no dia 01/04/2023?

Tendo em vista que o dataframe "df_chamados" já está filtrado pela data em questão, apenas se faz necessário contarmos quantas linhas ele possui. Deste modo, ao utilizarmos a função len no dataframe, conseguimos a resposta desejada.

In [15]:
len(df_chamados)

73

Resposta: 73 chamados

### 2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?


Para conseguirmos saber o tipo de chamado que teve mais chamados aberto no dia em questão, basta realizarmos a contagem de valores agregando a coluna tipo, e após isso selecionarmos o primeiro valor. Assim, filtrando o dataframe "df_chamados" pela coluna "tipo" e utilizando a função "value_counts", agregamos o dataframe pela coluna desejada e fazemos a contagem de linhas referentes a cada tipo. Por fim, a função "head" com o parâmetro "1", nos retorna o maior valor dessa filtragem.

In [23]:
df_chamados['tipo'].value_counts().head(1)

Poluição sonora    24
Name: tipo, dtype: int64

Resposta: O tipo de chamado com mais chamados abertos no dia 01/04/2023 foi Poluição Sonora, com 24 chamados abertos.

### 3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?


A fim de encontrarmos a resposta dessa questão, precisamos relacionar as tabelas de chamados e bairros, a fim de contarmos quais bairros possuem mais chamados abertos na data em questão. Assim, ao fazermos o "merge" dos dataframes "df_chamados" e "df_bairros", utilizando a coluna "id_bairro" presente em ambos, conseguimos um dataframe que contem os dados dos chamados abertos no dia 01/04/2023 com as informações referentes ao bairro do chamado. Logo, basta filtrar da mesma maneira mostrada na questão anterior, porém mudando o tipo para "nome", que é a coluna referente ao nome do bairro, e colocar o parâmentro da função "head" para "3", pegando assim os 3 bairros com mais chamados abertos no dia em questão.  

In [28]:
df_chamados_bairro = df_chamados.merge(df_bairros, on="id_bairro")

In [29]:
df_chamados_bairro['nome'].value_counts().head(3)

Engenho de Dentro    8
Leblon               6
Campo Grande         6
Name: nome, dtype: int64

Respostas: Os 3 bairros com mais chamados abertos na data em questão foram: Engenho de Dentro, com 8 chamados; Leblon, com 6 chamados; e Campo Grande, também com 6 chamados.

### 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

Para a resposta em questão, utilizaremos a mesma relação criada na resposta anterior, porém buscamos o nome da subprefeitura com mais chamados abertos no dia em questão, não os 3 bairros com mais chamados abertos nesse dia. Assim, basta mudarmos a filtragem da coluna "nome" para a coluna "subprefeitura", e mudarmos o parâmetro da função "head" para "1", conseguindo assim a subprefeitura com mais chamados abertos nesse dia.

In [32]:
df_chamados_bairro['subprefeitura'].value_counts().head(1)

Zona Norte    25
Name: subprefeitura, dtype: int64

Resposta: A subprefeitura com mais chamados abertos nesse dia foi a Zona Norte, com 25 chamados.

###  5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

Para essa questão, é necessário apenas contar quais registros de chamados no dataframe "df_chamados" não possuem valor na coluna "id_bairro", visto que isso impossibilita-os de terem um bairro associado. Assim, precisamos apenas utilizar a função "loc", filtrando a coluna "id_bairro" com a função "isna", que retorna os registros com valor nulo na coluna em questão. A partir disto, podemos inferir o motivo do chamado não ter bairro associado analisando as suas colunas. 

In [39]:
df_chamados.loc[df_chamados['id_bairro'].isna()]

,id_chamado,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,id_unidade_organizacional_mae,...,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,geometry
0,18516246,2023-04-01 00:55:38,2023-04-01 00:55:38,None,None,None,<NA>,1706,TR/SUBOP/CFT - Coordenadoria de Fiscalização e...,SMTR - Secretaria Municipal de Transportes,...,<NA>,D,F,No prazo,Encerrado,Atendido parcialmente,None,0,2023-04-01,None


Resposta: Existe 1 chamado que não possui id_bairro (id_chamado 18516246), logo, não possui bairro associado. Isso se deve pelo fato de ser um chamado do tipo "Ônibus", sendo assim, não possui necessariamente uma localização fixa para o chamado.
